# Main - Charcrete LCA

## 1. Set the project and import Charcrete database

In [15]:
# === Switch to kernel bw25! ===

# Brightway libraries
import bw2data as bd
import bw2io as bi
import bw2calc as bc
import bw2analyzer as ba
import bw_processing as bwp

# General libraries
import numpy as np
import pandas as pd
from fs.zipfs import ZipFS
import json                        # Library for working with json files
from pathlib import Path           # Library for working with paths in different OS     
import matplotlib.pyplot as plt    # Library for creating plots
import os

In [16]:
bd.projects.set_current("ei38-teaching-25")

### Importing Charcrete database Excel-file

In [17]:
if "Charcrete_38" in bd.databases:
    del bd.databases["Charcrete_38"]
else:
    # 1. Specify filepath to your foreground inventories.
    charcrete_path = os.getcwd()+"/data/Charcrete_database_38.xlsx"
    #charcrete_path = "/home/jupyter-mija.frossard/Charcrete-LCA/Water_database_38.xlsx"
    # 2. Create an instance of a class that contains basic methods for importing a database from an excel file.
    charcrete = bi.ExcelImporter(charcrete_path)  
    # 3. `apply_strategies` is one of such basic methods, it makes sure units, locations, etc are in correct format.
    charcrete.apply_strategies()
    # 4. Next step is to link your foreground exchanges to existing databases by matching relevant exchanges fields.
    charcrete.match_database("biosphere3", fields=("name", "unit", "categories"))
    charcrete.match_database("ei 3.8 cutoff", fields=("name", "location", "unit"))
    #charcrete.metadata.pop(None)  # Remove metadata None entry. TODO
    # 5. If everything is linked, write database so that it is saved in your project.
    if charcrete.all_linked:
        charcrete.write_database()

Extracted 1 worksheets in 0.01 seconds
Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 16 strategies in 7.27 seconds
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
5 datasets
23 exchanges
0 unlinked exchanges
  
Title: Writing activities to SQLite3 da

In [18]:
for unlinked in charcrete.unlinked:
    print(unlinked)

In [19]:
bd.databases

Databases dictionary with 8 object(s):
	Charcrete_38
	EXIOBASE 3.8.1 2017 monetary
	EXIOBASE 3.8.1 2017 monetary biosphere
	Korean Motors Chaebol
	biosphere3
	charcrete
	ei 3.8 cutoff
	강한 국가를 위한 영광스러운 구리

In [20]:
char_db = bd.Database("Charcrete_38")

In [21]:
 ei_db = bd.Database("ei 3.8 cutoff")

## 2. Basic calculations and comparisons

In [22]:
len(char_db)

5

In [23]:
[act for act in char_db]
charcrete = [act for act in char_db if "Charcrete" in act["name"]][0]
charcrete

'Charcrete production' (cubic meter, SE, None)

In [107]:
concrete_ei = [act for act in ei_db if "concrete production 20MPa" in act["name"]]
concrete_ei

['concrete production 20MPa' (cubic meter, North America without Quebec, None),
 'concrete production 20MPa' (cubic meter, RoW, None),
 'concrete production 20MPa' (cubic meter, CA-QC, None),
 'concrete production 20MPa' (cubic meter, ZA, None)]

In [108]:
concrete_ei.append(charcrete)
print(concrete_ei)

['concrete production 20MPa' (cubic meter, North America without Quebec, None), 'concrete production 20MPa' (cubic meter, RoW, None), 'concrete production 20MPa' (cubic meter, CA-QC, None), 'concrete production 20MPa' (cubic meter, ZA, None), 'Charcrete production' (cubic meter, SE, None)]


#### Choice of an LCIA method: IPCC 2013 GWP 100

In [109]:
ipcc = ('IPCC 2013', 'climate change', 'GWP 100a')
method = bd.Method(ipcc)

#### Comparison of the activities of biochar, charcrete and concrete production

In [110]:
def calculateLCA(act, amount, method):
    fu, data_objs, _ = bd.prepare_lca_inputs({act: amount}, method=ipcc)
    lca = bc.LCA(fu, data_objs=data_objs)
    lca.lci()
    lca.lcia()
    return lca.score

In [111]:
concretes_gwp = [calculateLCA(act, 1, ipcc) for act in concrete_ei]

In [112]:
pd.DataFrame(list(zip([act["name"] for act in concrete_ei],[act["location"] for act in concrete_ei], concretes_gwp)), columns=["activity", "location", "GWP"])

,activity,location,GWP
0,concrete production 20MPa,North America without Quebec,217.552437
1,concrete production 20MPa,RoW,193.853911
2,concrete production 20MPa,CA-QC,234.187992
3,concrete production 20MPa,ZA,229.719127
4,Charcrete production,SE,317.228383


Contribution analysis

In [117]:
ba.print_recursive_calculation(concrete_ei[1], ipcc, cutoff=0.025)

Fraction of score | Absolute score | Amount | Activity
0001 | 193.9 |     1 | 'concrete production 20MPa' (cubic meter, RoW, None)
  0.839 | 162.6 | 256.6 | 'market for cement, portland fly ash cement 21-35%' (kilogram, RoW, No
    00.8 |   155 | 256.6 | 'cement production, fly ash 21-35%' (kilogram, RoW, None)
      0.0296 | 5.739 | 6.118 | 'market group for electricity, medium voltage' (kilowatt hour, RAS, No
      0.146 | 28.27 | 30.21 | 'market for clinker' (kilogram, Europe without Switzerland, None)
      0.0303 | 5.868 | 6.363 | 'market for clinker' (kilogram, BR, None)
      0.129 | 24.98 | 28.23 | 'market for clinker' (kilogram, IN, None)
      0.43 | 83.42 | 90.59 | 'market for clinker' (kilogram, RoW, None)
    0.0318 | 6.173 | 44.72 | 'market for transport, freight, lorry, unspecified' (ton kilometer, Ro
  0.0792 | 15.36 |  1142 | 'market for sand' (kilogram, BR, None)
    0.0271 | 5.252 | 40.44 | 'market for transport, freight, lorry 16-32 metric ton, EURO3' (ton ki
      

In [119]:
ba.print_recursive_calculation(concrete_ei[2], ipcc, cutoff=0.025)

Fraction of score | Absolute score | Amount | Activity
0001 | 234.2 |     1 | 'concrete production 20MPa' (cubic meter, CA-QC, None)
  0.0483 |  11.3 |   950 | 'market for sand' (kilogram, RoW, None)
  0.0477 | 11.17 | 960.1 | 'market for gravel, round' (kilogram, RoW, None)
  0.878 | 205.6 | 239.5 | 'market for cement, Portland' (kilogram, CA-QC, None)
    0.855 | 200.2 | 239.5 | 'cement production, Portland' (kilogram, CA-QC, None)
      0.85 |   199 | 220.4 | 'clinker production' (kilogram, CA-QC, None)


In [115]:
ba.print_recursive_calculation(concrete_ei[4], ipcc, cutoff=0.025)

Fraction of score | Absolute score | Amount | Activity
0001 | 317.2 |     1 | 'Charcrete production' (cubic meter, SE, None)
  0.957 | 303.6 |   347 | 'market for cement, Portland' (kilogram, Europe without Switzerland, N
    0.934 | 296.2 |   347 | 'cement production, Portland' (kilogram, Europe without Switzerland, N
      0.914 | 289.8 | 313.2 | 'clinker production' (kilogram, Europe without Switzerland, None)
